In [1]:
import numpy as np
from pdb import set_trace

import quple
from quple.classifiers import QSVM, QSVMLogger
from quple.data_encoding import  EncodingCircuit
from quple import ParameterisedCircuit, PauliBlock
from quple.data_encoding.encoding_maps import trial_2
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from quple.components.data_preparation import prepare_train_val_test
#import tensorflow as tf
random_seed = 10598
# data preparation
#data = np.load('/afs/cern.ch/work/c/chlcheng/Repository/qml_work/data/ttH_hadronic_ATLAS.npy')
data = np.load('data/ttH_hadronic_ATLAS.npy')
x = data[:, :-1]
y = data[:, -1]

n_trials = 1
depth = 2

In [2]:
degree = 1
n_qubit = 20
n_event = 20000

In [3]:
if degree > 0:
    encoding_circuit = EncodingCircuit(feature_dimension=n_qubit, 
                                        copies=depth,
                                        rotation_blocks=['H',PauliBlock('Z', trial_2(1)),
                                                           PauliBlock('Y',trial_2(degree))],
                                        entanglement_blocks=[PauliBlock('ZZ',trial_2(degree))],
                                        entangle_strategy='alternate_linear')
else:
    encoding_circuit = EncodingCircuit(feature_dimension=n_qubit, 
                                        copies=depth,
                                        rotation_blocks=['H',PauliBlock('Z', trial_2(1))],
                                        entanglement_blocks=[PauliBlock('ZZ',trial_2(0))],
                                        entangle_strategy='alternate_linear')  

In [4]:
preprocessors = [PCA(n_components=n_qubit, random_state=3),  MinMaxScaler((-1,1))]
x_train, x_val, x_test, y_train, y_val, y_test = prepare_train_val_test(x, y, train_size=n_event, 
                                        val_size=n_event, test_size=n_event, 
                                        preprocessors=preprocessors, random_state=random_seed, stratify=y)              

In [5]:
qsvm = QSVM(encoding_circuit)

In [6]:
nrows, ncols = 2000, 2**n_qubit

In [7]:
interval = 1000

In [ ]:
for i in range(round(nrows/interval)):
    f = np.memmap('ttH_ATLAS_20_qubit_2000_events_train_statevectors.dat', dtype=np.complex64,
              mode='w+', shape=(nrows, ncols))
    print('Getting {}-th statevectors'.format(i))
    state_vectors = qsvm.get_state_vectors(x_train[i*interval:(i+1)*interval])
    print('Saving')
    f[i*interval:(i+1)*interval, :] = state_vectors
    print('Done')
    del f

In [17]:
for i in range(round(nrows/interval)):
    f = np.memmap('/home/chlcheng/ttH_ATLAS_20_qubit_2000_events_train_statevectors.dat', dtype=np.complex64,
              mode='w+', shape=(nrows, ncols))
    print('Getting {}-th statevectors'.format(i))
    state_vectors = qsvm.get_state_vectors(x_train[i*interval:(i+1)*interval])
    print('Saving')
    f[i*interval:(i+1)*interval, :] = state_vectors
    print('Done')
    del f

Getting 0-th statevectors
Saving
Done
Getting 1-th statevectors
Saving
Done


In [1]:
import numpy as np
f = np.memmap('/home/chlcheng/ttH_ATLAS_20_qubit_2000_events_train_statevectors.dat', dtype=np.complex64,
              mode='r', shape=(2000, 2**20))

In [ ]:
f = np.memmap('ttH_ATLAS_20_qubit_2000_events_train_statevectors.dat', dtype=np.complex64,
              mode='r', shape=(n_event, 2**n_qubit))

In [ ]:
qsvm.get_kernel_matrix(f, f)

In [9]:

def _block_slices(dim_size, block_size):
    """Generator that yields slice objects for indexing into 
    sequential blocks of an array along a particular axis
    """
    count = 0
    while True:
        yield slice(count, int(count + block_size), 1)
        count += block_size
        if count > dim_size:
            raise StopIteration

def blockwise_dot(A, B, max_elements=int(2**18), out=None):
    """
    Computes the dot product of two matrices in a block-wise fashion. 
    Only blocks of `A` with a maximum size of `max_elements` will be 
    processed simultaneously.
    """

    m,  n = A.shape
    n1, o = B.shape

    if n1 != n:
        raise ValueError('matrices are not aligned')

    if A.flags.f_contiguous:
        # prioritize processing as many columns of A as possible
        max_cols = int(max(1, max_elements / m))
        max_rows =  int(max_elements / max_cols)

    else:
        # prioritize processing as many rows of A as possible
        max_rows = int(max(1, max_elements / n))
        max_cols = int(max_elements / max_rows)

    if out is None:
        out = np.empty((m, o), dtype=np.result_type(A, B))
    elif out.shape != (m, o):
        raise ValueError('output array has incorrect dimensions')
    
    for mm in _block_slices(m, max_rows):
        print(mm)
        out[mm, :] = 0
        for nn in _block_slices(n, max_cols):
            print(nn)
            A_block = A[mm, nn].copy()  # copy to force a read
            out[mm, :] += A_block @ B[nn, :]
            del A_block

    return out

In [ ]:
result = blockwise_dot(f.conjugate(), f.T, max_elements=int(2**21))

slice(0, 2, 1)
slice(0, 1048576, 1)


In [4]:
result = f.conjugate() @ f.T